In [31]:
import pandas as pd
import numpy as np
from TM1py.Services import TM1Service
from TM1py.Utils import Utils
import calendar

#### 获取ZL Stock Strategy数据

In [80]:
tm1 = TM1Service(address="localhost", port=30059, ssl=False, user="admin", password="")

In [6]:
cube_src="ZL Stock Strategy"
vSecenario="ACT"
vVersion="WIP"
vCOACompany="002"
vCountry="US"
vProduct="LSKU1571"
mdx='''
        SELECT 
          NON EMPTY 
           {[M ZL Stock Strategy].[Sales Price Per Unit],[M ZL Stock Strategy].[Cost Per Unit],[M ZL Stock Strategy].[Fee Rate Per Day]
           ,[M ZL Stock Strategy].[Product Volume],[M ZL Stock Strategy].[Mean Sales Units],[M ZL Stock Strategy].[Standard Deviation]}
          ON COLUMNS , 
          NON EMPTY 
           {[SA Year].[SA Year].Members}
           * {[SA Month].[SA Month].Members}
          ON ROWS 
        FROM ['''+cube_src+'''] 
        WHERE 
          (
           [Scenario].[Scenario].['''+vSecenario+'''],
           [Version].[Version].['''+vVersion+'''],
           [COA Company].[COA Company].['''+vCOACompany+'''],
           [Country].[Country].['''+vCountry+'''],
           [Product].[Product].['''+vProduct+''']
          )

         '''
 #元组字典
cells=tm1.cubes.cells.execute_mdx(mdx,skip_rule_derived_cells=False,skip_zeros=True,skip_consolidated_cells=True)
df = Utils.build_pandas_dataframe_from_cellset(cells)


In [42]:
df2=df.reset_index().pivot(index=["Scenario","SA Year","SA Month","Version","COA Company","Country","Product"],columns=["M ZL Stock Strategy"],values="Values")

In [55]:
df2=df2.reset_index()

In [56]:
df2

M ZL Stock Strategy,index,Scenario,SA Year,SA Month,Version,COA Company,Country,Product,Cost Per Unit,Fee Rate Per Day,Mean Sales Units,Product Volume,Sales Price Per Unit,Standard Deviation
0,0,ACT,2021,M01,WIP,002,US,LSKU1571,5.0,0.5,1000.0,6.0,10.0,300.0


In [65]:
df2["Days of Month"]=calendar.monthrange(int(df2["SA Year"]),int(df2["SA Month"].str.slice(1,3)))[1]

In [72]:
df2

M ZL Stock Strategy,index,Scenario,SA Year,SA Month,Version,COA Company,Country,Product,Cost Per Unit,Fee Rate Per Day,Mean Sales Units,Product Volume,Sales Price Per Unit,Standard Deviation,Days of Month
0,0,ACT,2021,M01,WIP,002,US,LSKU1571,5.0,0.5,1000.0,6.0,10.0,300.0,31


In [99]:
for i in range(len(df2)):
    
    product_cost=df2["Cost Per Unit"][i]
    product_selling_price=df2["Sales Price Per Unit"][i]
    fee_rate_per=df2["Fee Rate Per Day"][i]
    product_volume=df2["Product Volume"][i]
    mean_sales=df2["Mean Sales Units"][i]
    standard_deviation=df2["Standard Deviation"][i]
    Days=df2["Days of Month"][i]
    
    cube_tar="ZL Stock Simulation Analysis"
    vSecenario=df2["Scenario"][i]
    vVersion=df2["Version"][i]
    vCOACompany=df2["COA Company"][i]
    vRegion=df2["Country"][i]
    vProduct=df2["Product"][i]
    vSA_Year=df2["SA Year"][i]
    vBusiness_unit="hangzhou"
    vCurrency="USD"
    vChannel="Amazon"
    vDatasource="Base"
    sales_units_simulation=round(pd.Series(np.random.normal(mean_sales,standard_deviation,Days)),0)
    
    mdx='''
        SELECT 
          
           {[M ZL Stock Simulation Analysis].[Sales Units - Normal distribution]}
          ON COLUMNS , 
          
           {Descendants([Month Day].['''+vSA_Month+'''])}
          ON ROWS 
        FROM ['''+cube_tar+'''] 
        WHERE 
          (
           [SA Year].['''+vSA_Year+'''],
           [Scenario].[Scenario].['''+vSecenario+'''],
           [Version].[Version].['''+vVersion+'''],
           [COA Company].[COA Company].['''+vCOACompany+'''],
           [Region].[Region].['''+vRegion+'''],
           [Product].[Product].['''+vProduct+'''],
           [Channel].[Channel].['''+vChannel+'''],
           [Currency].[Currency].['''+vCurrency+'''],
           [Business Unit].[Business Unit].['''+vBusiness_unit+'''],
           [Data Source ZL Stock Simulation Analysis].[Data Source ZL Stock Simulation Analysis].['''+vDatasource+''']
           
          )

         '''
     #元组字典
    cells=tm1.cubes.cells.execute_mdx(mdx,skip_rule_derived_cells=False,skip_zeros=False,skip_consolidated_cells=True)
    df3= Utils.build_pandas_dataframe_from_cellset(cells)
    df3=df3.reset_index()
    df3["Values"]=sales_units_simulation
    #clear
    tm1.cells.clear_with_mdx(cube_tar,mdx)
    #Writing to tgr Cube
    tm1.cells.write_dataframe(
        cube_name=cube_tar,
        data=df3,
        dimensions=["Scenario","SA Year","Month Day","Version","COA Company","Business Unit","Product","Currency","Region","Channel"
                    ,"Data Source ZL Stock Simulation Analysis","M ZL Stock Simulation Analysis"],
        increment=False,
        deactivate_transaction_log=True,
        reactivate_transaction_log=True,
        sandbox_name=None,
        use_ti=True,
        skip_non_updateable=True,
        use_changeset=False)

In [100]:
df3

,Scenario,SA Year,Month Day,Version,COA Company,Business Unit,Product,Currency,Region,Channel,Data Source ZL Stock Simulation Analysis,M ZL Stock Simulation Analysis,Values
0,ACT,2021,01-01,WIP,002,hangzhou,LSKU1571,USD,US,Amazon,Base,Sales Units - Normal distribution,870.0
1,ACT,2021,01-02,WIP,002,hangzhou,LSKU1571,USD,US,Amazon,Base,Sales Units - Normal distribution,1149.0
2,ACT,2021,01-03,WIP,002,hangzhou,LSKU1571,USD,US,Amazon,Base,Sales Units - Normal distribution,894.0
3,ACT,2021,01-04,WIP,002,hangzhou,LSKU1571,USD,US,Amazon,Base,Sales Units - Normal distribution,962.0
4,ACT,2021,01-05,WIP,002,hangzhou,LSKU1571,USD,US,Amazon,Base,Sales Units - Normal distribution,959.0
5,ACT,2021,01-06,WIP,002,hangzhou,LSKU1571,USD,US,Amazon,Base,Sales Units - Normal distribution,1059.0
6,ACT,2021,01-07,WIP,002,hangzhou,LSKU1571,USD,US,Amazon,Base,Sales Units - Normal distribution,1374.0
7,ACT,2021,01-08,WIP,002,hangzhou,LSKU1571,USD,US,Amazon,Base,Sales Units - Normal distribution,1127.0
8,ACT,2021,01-09,WIP,002,hangzhou,LSKU1571,USD,US,Amazon,Base,Sales Units - Normal distribution,1064.0
9,ACT,2021,01-10,WIP,002,hangzhou,LSKU1571,USD,US,Amazon,Base,Sales Units - Normal distribution,763.0
